In [1]:
# 2 = full
# 3 = partial


import cv2
from gentraining import *
import keras.models
import matplotlib.pyplot as plt
from collections import Counter
from editdistance import EditDistanceFinder

model2 = keras.models.load_model('../model/sanborn_rotnet2.hdf5')
model3 = keras.models.load_model('../model/sanborn_rotnet3.hdf5')
print('models loaded')

H = 48
W = 72
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
test_letter_set = 'ABCDEFGHIKLMNOPRSTUVW'
character_set = set([i for i in test_letter_set])

# load data
letter_path = "../data/letter/"
letter_files = [f for f in listdir(letter_path) if isfile(join(letter_path, f))]
letter_keys = [f[len(f)-5] for f in letter_files]

training_images = []
for f in letter_files:
    img = crop_width(tightest_crop(pad_image(resize_preserve_aspect_ratio(cv2.imread(letter_path + f),W,H),W,H),H),W,H)
    _,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    training_images.append(img)
# todo: split into train/test letters
print("there are %d traning examples" % len(training_images))

"""
# look at individual training examples
for i in range(10):
    img,char_list,start_list,stop_list = generate_training_example(training_images, 
                    letter_keys, character_set=None, full_widths_only=False,
                    W=72, H=48)
    print(char_list)
    print(start_list)
    print(stop_list)

    X = np.zeros((1, H, W, 1))
    X[0,:,:,0] = np.array(img[:,:,0])
    answer = model3.predict(X).flatten()
    answer = [(letters[i-3], round(answer[i],3)) for i in range(3,27)]
    answer = sorted(answer, key = lambda x: -x[1])
    print(answer)

    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
"""

def newBatch(n):
    return np.zeros((n, H, W, 1))

def addToBatch(X, i, img, x1, x2):
    st = int((72 - (x2 - x1))/2)
    if st < 0: st = 0
    end = st + x2 - x1
    if end > W: end = W
    if x2 - x1 > W: x2 = x1 + W
    X[i,0:H,st:end,0] = np.array(img[0:H,x1:x2,0])

def predictBatch(X, model):
    return model.predict(X) 

def predictionRowToAnswerTuples(y):
    answer = [(letters[i-3], round(y[i],3)) for i in range(3,29)]
    answer = sorted(answer, key = lambda x: -x[1])
    return answer


def predict2(model2, img, x1, x2):
    X = np.zeros((1, H, W, 1))
    st = int((72 - (x2 - x1))/2)
    if st < 0: st = 0
    end = st + x2 - x1
    #print("%d %d %d %d " % (st, end, x1, x2))
    X[0,0:H,st:end,0] = np.array(img[0:H,x1:x2,0])
    answer = model2.predict(X).flatten()
    answer = [(letters[i-3], round(answer[i],3)) for i in range(3,29)]
    answer = sorted(answer, key = lambda x: -x[1])
    return answer # sorted list of (letter, score) tuples

def predict3(model3, img, x1, x2):
    X = np.zeros((1, H, W, 1))
    st = int((72 - (x2 - x1))/2)
    if st < 0: st = 0
    end = st + x2 - x1
    X[0,0:H,st:end,0] = np.array(img[0:H,x1:x2,0])
    answer = model3.predict(X).flatten()
    answer = [(letters[i-3], round(answer[i],3)) for i in range(3,29)]
    answer = sorted(answer, key = lambda x: -x[1])
    return answer # sorted list of (letter, score) tuples


C:\Python36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


models loaded
there are 481 traning examples


In [ ]:
def slidingWindow(model2, img):
    """  """
    
    w = img.shape[1]
    stepSize = 8 # how much space between images that are checked?
    # we require that W/stepSize is an integer.

    # table holds individual letter probs for each slice
    numSlices = int(w/stepSize) + int(W/stepSize)
    table = [[0.0 for i in range(26)] for j in range(numSlices)]

    n = 0
    for x1 in range(-W+20,w-20):
        if x1/stepSize != int(x1/stepSize):
            continue

        n += 1
        x2 = x1 + W
        idx = int((x1+W)/stepSize)
        
        x1 = max(0,x1)
        x2 = min(w,x2)

        answer = predict2(model2, img, x1, x2)
        #bestLetter = str(answer[0][0])
        #bestProb = answer[0][1] / sum([answer[i][1] for i in range(len(answer))])
        print("--pred for %d %d" % (x1,x2))
        dictanswer = dict(answer)
        y = [dictanswer[l] for l in letters ]
        table[idx][0:26] = y

    for c in range(26):
        plt.plot(range(n), [table[i][c] for i in range(n)], label=chr(65+c))
    plt.legend()
    plt.show()

    return 0

In [20]:
def dynamic(model3, img):
    """ returns (word, confidence, starts, ends) """
    
    printInfo = True
    w = img.shape[1]
    spacingWindow = 8 # how much freedom is allowed in the spacing?
    stepSize = 4 # how much space between images that are checked?
    # we require that W/stepSize is an integer.

    # DP table holds (score, subword, charStart, charEnd) 
    # for the subimage from 0:i
    table = [(-1,"",-1,-1) for i in range(w)]

    batchSize = 0
    for x2 in range(0,w,stepSize):
        for x1 in range(max(0,x2-W+1), x2):
            if x1/stepSize != int(x1/stepSize):
                continue
            batchSize += 1
    X = newBatch(batchSize)

    batchIdx = 0
    for x2 in range(0,w,stepSize):
        for x1 in range(max(0,x2-W+1), x2):
            if x1/stepSize != int(x1/stepSize):
                continue

            addToBatch(X, batchIdx, img, x1, x2+1)
            
            batchIdx += 1

    y = predictBatch(X, model3)

    batchIdx = 0
    for x2 in range(0,w,stepSize):
        for x1 in range(max(0,x2-W+1), x2):
            if x1/stepSize != int(x1/stepSize):
                continue

            answer = predictionRowToAnswerTuples(y[batchIdx])
            batchIdx += 1

            bestLetter = str(answer[0][0])
            bestProbNormalized = answer[0][1] / sum([answer[i][1] for i in range(len(answer))])
            bestProb = min(answer[0][1], bestProbNormalized)
            
            if x2-x1 <= 16:
                '''Iweight = 0
                for i in range(26):
                    if answer[i][0] == 'I':
                        Iweight = answer[i][1]
                if bestLetter == 'I' and Iweight > 0.15 and answer[1][1] < 0.1:
                    bestLetter = 'I'
                    bestProb = 1.00
                elif bestLetter == 'I' and Iweight > 0.15 and answer[1][1] < 0.2:
                    bestLetter = 'I'
                    bestProb = 0.80
                elif bestLetter != 'I' and answer[0][1] < 0.3 and Iweight > 0.15:
                    bestLetter = 'I'
                    bestProb = 0.60'''
                if answer[0][0] == 'I':
                    ratio = answer[0][1]/(answer[1][1]+0.0001)
                    if answer[1][1] < 0.4 and answer[0][1] > 0.1:
                        if ratio > 3:
                            bestProb = 1
                        elif ratio > 2:
                            bestProb = 0.97
                        elif ratio > 1.5:
                            bestProb = 0.93
                        elif ratio >= 1:
                            bestProb = 0.87
                elif answer[1][0] == 'I':
                    ratio = answer[1][1]/(answer[0][1]+0.0001)
                    if answer[0][1] < 0.3:
                        if ratio > 0.5 and answer[1][1] > 0.2:
                            bestLetter = 'I'
                            bestProb = 0.87
                        elif ratio > 0.5 and answer[1][1] > 0.1:
                            bestLetter = 'I'
                            bestProb = 0.74

            if x2-x1 > 48 and bestLetter not in 'MW':
                continue
            if x2-x1 > 16 and bestLetter in 'I':
                continue

            debugst = [answer[i][0] + (" %.2f " % answer[i][1]) for i in range(5) ]
            if printInfo: print("--pred for %d %d %s %.3f %s" % (x1,x2,bestLetter,bestProb,debugst))
            

            bestPrevString = ""
            bestPrevProb = -1
            bestPrevStarts = []
            bestPrevEnds = []

            for xp in range(max(0, int(x1-spacingWindow/2)), min(x2, x1+spacingWindow)):
                if xp/stepSize != int(xp/stepSize):
                    continue # ensure evenly divisible by step size

                prevProb, prevString, prevStarts, prevEnds = table[xp]
                if prevProb > bestPrevProb:
                    bestPrevProb = prevProb
                    bestPrevString = prevString
                    bestPrevStarts = prevStarts
                    bestPrevEnds = prevEnds

            if x1 == 0:
                bestPrevString = ""
                bestPrevProb = 1
                bestPrevStarts = []
                bestPrevEnds = []

            if bestProb * bestPrevProb > table[x2][0]:
                table[x2] = (bestProb*bestPrevProb, bestPrevString+bestLetter, 
                            bestPrevStarts + [x1], bestPrevEnds + [x2])
        
        if printInfo: print("%d %.3f %s" % (x2, table[x2][0], table[x2][1]))    

    answerLoc = w
    while answerLoc/stepSize != int(answerLoc/stepSize) or answerLoc >= len(table):
        answerLoc -= 1
    if printInfo: print(answerLoc)
    if printInfo: print(len(table))
    return table[answerLoc]

In [21]:

e = EditDistanceFinder()
cnt = Counter()
showImages = False
numCorrect = 0
runningCount = 0
sumED = 0
numPredictions = 10

# test approaches
for i in range(numPredictions):
    img,char_list,char_starts,char_ends = generate_test_example(
                    training_images, letter_keys, character_set=character_set,
                    H=48, min_chars=2, max_chars=6)
    true_word = ''.join(char_list)
    

    
    conf3, word3, pred_starts, pred_ends = dynamic(model3, img)
    
    print('New example, correct answers:')
    print(''.join(char_list))
    print(char_starts)
    print(char_ends)
    
    print("model 3: %s %.3f" % (word3, conf3))
    print(pred_starts)
    print(pred_ends)

    dist,alignment = e.align(word3, true_word)
    print("edit distance: %d" % dist)
    sumED += dist
    for a in alignment:
        cnt[a] += 1
    if dist == 0:
        numCorrect += 1
    runningCount += 1
 
    print("Running tally::: correct: %d total: %d accuracy: %.3f avgEditDist %.3f" % (numCorrect,
    runningCount,float(numCorrect)/runningCount,
    float(sumED)/runningCount))

    #slidingWindow(model2, img)
    if showImages:
        cv2.imshow('image',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

for a in cnt.keys():
    if a[0] != a[1]:
        print("-------", end="")
    print("%s: %d" % (a, cnt[a]))



0 -1.000 
--pred for 0 4 O 0.065 ['O 0.06 ', 'C 0.04 ', 'G 0.03 ', 'A 0.01 ', 'D 0.00 ']
4 0.065 O
--pred for 0 8 O 0.096 ['O 0.10 ', 'C 0.08 ', 'G 0.07 ', 'D 0.00 ', 'Y 0.00 ']
--pred for 4 8 O 0.069 ['O 0.07 ', 'C 0.04 ', 'G 0.04 ', 'D 0.00 ', 'L 0.00 ']
8 0.096 O
--pred for 0 12 O 0.191 ['O 0.19 ', 'C 0.10 ', 'G 0.06 ', 'D 0.01 ', 'V 0.01 ']
--pred for 4 12 O 0.120 ['O 0.12 ', 'C 0.06 ', 'G 0.04 ', 'D 0.00 ', 'A 0.00 ']
--pred for 8 12 O 0.034 ['O 0.03 ', 'V 0.03 ', 'S 0.02 ', 'C 0.02 ', 'R 0.02 ']
12 0.191 O
--pred for 0 16 O 0.283 ['O 0.28 ', 'C 0.14 ', 'G 0.04 ', 'D 0.01 ', 'V 0.01 ']
--pred for 4 16 O 0.222 ['O 0.22 ', 'C 0.09 ', 'G 0.03 ', 'D 0.01 ', 'V 0.00 ']
--pred for 8 16 O 0.167 ['O 0.17 ', 'C 0.06 ', 'G 0.02 ', 'D 0.01 ', 'V 0.01 ']
--pred for 12 16 O 0.067 ['O 0.07 ', 'C 0.03 ', 'G 0.01 ', 'D 0.01 ', 'E 0.01 ']
16 0.283 O
--pred for 0 20 O 0.361 ['O 0.36 ', 'C 0.15 ', 'G 0.02 ', 'D 0.01 ', 'V 0.01 ']
--pred for 4 20 O 0.296 ['O 0.30 ', 'C 0.12 ', 'G 0.02 ', 'D 0.01 ', '

0 -1.000 
--pred for 0 4 T 0.085 ['T 0.09 ', 'V 0.03 ', 'W 0.01 ', 'Y 0.01 ', 'F 0.01 ']
4 0.085 T
--pred for 0 8 T 0.208 ['T 0.21 ', 'F 0.01 ', 'A 0.00 ', 'P 0.00 ', 'V 0.00 ']
--pred for 4 8 T 0.115 ['T 0.12 ', 'V 0.01 ', 'F 0.01 ', 'Y 0.01 ', 'A 0.00 ']
8 0.208 T
--pred for 0 12 T 0.296 ['T 0.30 ', 'F 0.01 ', 'P 0.00 ', 'A 0.00 ', 'R 0.00 ']
--pred for 4 12 T 0.220 ['T 0.22 ', 'F 0.01 ', 'P 0.00 ', 'Y 0.00 ', 'A 0.00 ']
--pred for 8 12 T 0.111 ['T 0.11 ', 'Y 0.01 ', 'V 0.01 ', 'F 0.01 ', 'A 0.00 ']
12 0.296 T
--pred for 0 16 T 0.363 ['T 0.36 ', 'F 0.01 ', 'P 0.01 ', 'A 0.00 ', 'R 0.00 ']
--pred for 4 16 T 0.316 ['T 0.32 ', 'F 0.01 ', 'P 0.01 ', 'A 0.00 ', 'R 0.00 ']
--pred for 8 16 T 0.217 ['T 0.22 ', 'F 0.01 ', 'P 0.00 ', 'V 0.00 ', 'Y 0.00 ']
--pred for 12 16 T 0.116 ['T 0.12 ', 'V 0.01 ', 'F 0.01 ', 'Y 0.01 ', 'A 0.00 ']
16 0.363 T
--pred for 0 20 T 0.414 ['T 0.41 ', 'F 0.01 ', 'P 0.00 ', 'A 0.00 ', 'C 0.00 ']
--pred for 4 20 T 0.371 ['T 0.37 ', 'F 0.01 ', 'P 0.00 ', 'V 0.00 ', '

0 -1.000 
--pred for 0 4 V 0.087 ['V 0.09 ', 'Y 0.03 ', 'W 0.01 ', 'T 0.01 ', 'C 0.00 ']
4 0.087 V
--pred for 0 8 V 0.212 ['V 0.21 ', 'W 0.01 ', 'Y 0.01 ', 'F 0.00 ', 'M 0.00 ']
--pred for 4 8 V 0.081 ['V 0.08 ', 'W 0.03 ', 'M 0.01 ', 'N 0.01 ', 'H 0.00 ']
8 0.212 V
--pred for 0 12 V 0.296 ['V 0.30 ', 'W 0.02 ', 'T 0.01 ', 'Y 0.01 ', 'C 0.00 ']
--pred for 4 12 V 0.101 ['V 0.10 ', 'W 0.10 ', 'M 0.00 ', 'T 0.00 ', 'Y 0.00 ']
--pred for 8 12 V 0.058 ['V 0.06 ', 'W 0.05 ', 'M 0.02 ', 'I 0.01 ', 'N 0.00 ']
12 0.296 V
--pred for 0 16 V 0.338 ['V 0.34 ', 'W 0.05 ', 'D 0.01 ', 'Y 0.01 ', 'O 0.00 ']
--pred for 4 16 W 0.158 ['W 0.16 ', 'V 0.12 ', 'M 0.01 ', 'Y 0.00 ', 'D 0.00 ']
--pred for 8 16 V 0.108 ['V 0.11 ', 'W 0.11 ', 'M 0.01 ', 'L 0.00 ', 'R 0.00 ']
--pred for 12 16 A 0.085 ['A 0.09 ', 'V 0.03 ', 'W 0.01 ', 'I 0.01 ', 'M 0.01 ']
16 0.338 V
--pred for 0 20 V 0.506 ['V 0.51 ', 'W 0.01 ', 'D 0.01 ', 'C 0.00 ', 'S 0.00 ']
--pred for 4 20 V 0.380 ['V 0.38 ', 'W 0.04 ', 'M 0.01 ', 'D 0.00 ', '

--pred for 88 124 T 0.845 ['T 0.85 ', 'V 0.01 ', 'F 0.00 ', 'R 0.00 ', 'I 0.00 ']
--pred for 92 124 T 0.769 ['T 0.77 ', 'V 0.01 ', 'F 0.00 ', 'I 0.00 ', 'O 0.00 ']
--pred for 96 124 T 0.687 ['T 0.69 ', 'V 0.01 ', 'O 0.00 ', 'A 0.00 ', 'D 0.00 ']
--pred for 100 124 T 0.597 ['T 0.60 ', 'A 0.00 ', 'F 0.00 ', 'I 0.00 ', 'L 0.00 ']
--pred for 104 124 T 0.494 ['T 0.49 ', 'F 0.01 ', 'I 0.01 ', 'A 0.00 ', 'L 0.00 ']
--pred for 108 124 T 0.400 ['T 0.40 ', 'F 0.00 ', 'D 0.00 ', 'I 0.00 ', 'L 0.00 ']
--pred for 112 124 T 0.298 ['T 0.30 ', 'F 0.01 ', 'P 0.00 ', 'L 0.00 ', 'R 0.00 ']
--pred for 116 124 T 0.221 ['T 0.22 ', 'F 0.01 ', 'L 0.00 ', 'P 0.00 ', 'R 0.00 ']
--pred for 120 124 T 0.124 ['T 0.12 ', 'F 0.01 ', 'V 0.01 ', 'Y 0.00 ', 'A 0.00 ']
124 0.745 ASIT
124
127
New example, correct answers:
ASIT
[0, 40, 76, 83]
[42, 77, 85, 127]
model 3: ASIT 0.745
[0, 40, 76, 84]
[40, 76, 80, 124]
edit distance: 0
Running tally::: correct: 4 total: 4 accuracy: 1.000 avgEditDist 0.000
0 -1.000 
--pred for 0

--pred for 48 80 B 0.744 ['B 0.75 ', 'E 0.08 ', 'I 0.05 ', 'N 0.03 ', 'L 0.02 ']
--pred for 52 80 B 0.683 ['B 0.68 ', 'I 0.06 ', 'E 0.06 ', 'N 0.04 ', 'R 0.02 ']
--pred for 56 80 B 0.583 ['B 0.58 ', 'E 0.05 ', 'I 0.05 ', 'L 0.03 ', 'N 0.02 ']
--pred for 60 80 B 0.466 ['B 0.47 ', 'I 0.05 ', 'E 0.04 ', 'N 0.03 ', 'R 0.01 ']
--pred for 64 80 B 0.390 ['B 0.39 ', 'I 0.05 ', 'E 0.04 ', 'N 0.03 ', 'L 0.02 ']
--pred for 68 80 B 0.272 ['B 0.27 ', 'I 0.07 ', 'N 0.05 ', 'E 0.03 ', 'L 0.02 ']
--pred for 72 80 R 0.057 ['R 0.06 ', 'C 0.04 ', 'I 0.04 ', 'L 0.03 ', 'E 0.03 ']
--pred for 76 80 N 0.055 ['N 0.05 ', 'I 0.04 ', 'E 0.02 ', 'L 0.01 ', 'R 0.01 ']
80 0.736 EIB
--pred for 36 84 B 0.741 ['B 0.97 ', 'E 0.09 ', 'L 0.08 ', 'I 0.05 ', 'N 0.04 ']
--pred for 40 84 B 0.732 ['B 0.94 ', 'E 0.10 ', 'I 0.07 ', 'N 0.05 ', 'L 0.04 ']
--pred for 44 84 B 0.712 ['B 0.86 ', 'E 0.11 ', 'I 0.07 ', 'N 0.04 ', 'L 0.03 ']
--pred for 48 84 B 0.689 ['B 0.75 ', 'E 0.09 ', 'I 0.07 ', 'N 0.05 ', 'L 0.04 ']
--pred for 52 8

0 -1.000 
--pred for 0 4 I 1.000 ['I 0.11 ', 'E 0.02 ', 'N 0.01 ', 'R 0.01 ', 'D 0.01 ']
4 1.000 I
--pred for 0 8 I 1.000 ['I 0.45 ', 'N 0.03 ', 'M 0.02 ', 'H 0.02 ', 'R 0.01 ']
--pred for 4 8 I 1.000 ['I 0.40 ', 'N 0.03 ', 'H 0.02 ', 'M 0.01 ', 'T 0.01 ']
8 1.000 I
--pred for 0 12 N 0.361 ['N 0.36 ', 'M 0.02 ', 'I 0.01 ', 'A 0.00 ', 'F 0.00 ']
--pred for 4 12 N 0.269 ['N 0.27 ', 'M 0.01 ', 'I 0.01 ', 'Y 0.01 ', 'A 0.00 ']
--pred for 8 12 N 0.093 ['N 0.09 ', 'M 0.01 ', 'I 0.01 ', 'C 0.01 ', 'V 0.01 ']
12 0.361 N
--pred for 0 16 N 0.466 ['N 0.47 ', 'I 0.01 ', 'M 0.01 ', 'L 0.00 ', 'A 0.00 ']
--pred for 4 16 N 0.360 ['N 0.36 ', 'Y 0.01 ', 'I 0.01 ', 'M 0.00 ', 'F 0.00 ']
--pred for 8 16 N 0.225 ['N 0.22 ', 'C 0.01 ', 'Y 0.01 ', 'M 0.00 ', 'O 0.00 ']
--pred for 12 16 N 0.130 ['N 0.13 ', 'Y 0.02 ', 'C 0.00 ', 'M 0.00 ', 'V 0.00 ']
16 0.466 N
--pred for 0 20 N 0.545 ['N 0.55 ', 'I 0.01 ', 'L 0.00 ', 'T 0.00 ', 'F 0.00 ']
--pred for 4 20 N 0.456 ['N 0.46 ', 'I 0.01 ', 'T 0.00 ', 'Y 0.00 ', '

0 -1.000 
--pred for 0 4 Y 0.089 ['Y 0.09 ', 'V 0.06 ', 'W 0.01 ', 'C 0.00 ', 'N 0.00 ']
4 0.089 Y
--pred for 0 8 V 0.225 ['V 0.22 ', 'Y 0.01 ', 'W 0.01 ', 'M 0.00 ', 'T 0.00 ']
--pred for 4 8 V 0.065 ['V 0.06 ', 'N 0.03 ', 'W 0.02 ', 'M 0.01 ', 'Y 0.00 ']
8 0.225 V
--pred for 0 12 V 0.333 ['V 0.33 ', 'W 0.01 ', 'Y 0.01 ', 'D 0.00 ', 'M 0.00 ']
--pred for 4 12 V 0.177 ['V 0.18 ', 'W 0.01 ', 'M 0.01 ', 'N 0.01 ', 'Y 0.00 ']
--pred for 8 12 V 0.082 ['V 0.08 ', 'M 0.03 ', 'W 0.02 ', 'N 0.01 ', 'D 0.00 ']
12 0.333 V
--pred for 0 16 V 0.405 ['V 0.41 ', 'W 0.01 ', 'D 0.01 ', 'O 0.01 ', 'Y 0.00 ']
--pred for 4 16 V 0.252 ['V 0.25 ', 'W 0.02 ', 'M 0.02 ', 'N 0.01 ', 'D 0.01 ']
--pred for 8 16 V 0.139 ['V 0.14 ', 'M 0.03 ', 'N 0.03 ', 'W 0.01 ', 'D 0.00 ']
--pred for 12 16 A 0.074 ['A 0.07 ', 'V 0.03 ', 'M 0.01 ', 'W 0.01 ', 'N 0.01 ']
16 0.405 V
--pred for 0 20 V 0.495 ['V 0.50 ', 'D 0.01 ', 'W 0.01 ', 'O 0.01 ', 'Y 0.00 ']
--pred for 4 20 V 0.372 ['V 0.37 ', 'W 0.01 ', 'M 0.01 ', 'D 0.00 ', '

0 -1.000 
--pred for 0 4 V 0.064 ['V 0.06 ', 'T 0.03 ', 'Y 0.02 ', 'W 0.01 ', 'A 0.00 ']
4 0.064 V
--pred for 0 8 V 0.104 ['V 0.10 ', 'W 0.03 ', 'Y 0.02 ', 'T 0.01 ', 'N 0.01 ']
--pred for 4 8 V 0.039 ['V 0.04 ', 'N 0.04 ', 'W 0.02 ', 'P 0.01 ', 'Y 0.01 ']
8 0.104 V
--pred for 0 12 V 0.289 ['V 0.29 ', 'W 0.01 ', 'D 0.01 ', 'M 0.00 ', 'Y 0.00 ']
--pred for 4 12 V 0.178 ['V 0.18 ', 'W 0.02 ', 'M 0.01 ', 'D 0.00 ', 'N 0.00 ']
--pred for 8 12 V 0.082 ['V 0.08 ', 'M 0.02 ', 'W 0.01 ', 'N 0.01 ', 'Y 0.00 ']
12 0.289 V
--pred for 0 16 V 0.394 ['V 0.39 ', 'W 0.01 ', 'D 0.01 ', 'T 0.01 ', 'Y 0.01 ']
--pred for 4 16 V 0.253 ['V 0.25 ', 'W 0.02 ', 'Y 0.02 ', 'M 0.01 ', 'P 0.01 ']
--pred for 8 16 V 0.160 ['V 0.16 ', 'M 0.03 ', 'Y 0.01 ', 'W 0.01 ', 'N 0.01 ']
--pred for 12 16 V 0.060 ['V 0.06 ', 'W 0.02 ', 'M 0.02 ', 'A 0.02 ', 'Y 0.01 ']
16 0.394 V
--pred for 0 20 V 0.498 ['V 0.50 ', 'D 0.01 ', 'W 0.01 ', 'M 0.00 ', 'O 0.00 ']
--pred for 4 20 V 0.400 ['V 0.40 ', 'M 0.02 ', 'W 0.01 ', 'B 0.01 ', '